In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [50]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import explode, desc

# SparkSession 초기화
spark = SparkSession.builder \
    .appName("Word Frequency Analysis") \
    .getOrCreate()

# assembled_data.csv 파일을 읽어서 DataFrame 생성
file_path = "bigdata.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# null 값을 가진 행 제거
df = df.dropna(subset=["comment"])

# Tokenizer를 사용하여 단어 추출
tokenizer = Tokenizer(inputCol="comment", outputCol="words")
tokenized_df = tokenizer.transform(df)

# 각 단어의 빈도수 계산
word_count_df = tokenized_df.select(explode("words").alias("word")).groupBy("word").count()

# 빈도수를 기준으로 내림차순 정렬
sorted_word_count_df = word_count_df.orderBy(desc("count"))

# 상위 100개 단어만 출력
sorted_word_count_df.show(100, truncate=False)


+----------+-----+
|word      |count|
+----------+-----+
|          |2415 |
|더        |506  |
|다        |410  |
|플라스틱  |383  |
|진짜      |347  |
|그냥      |328  |
|일회용품  |267  |
|왜        |255  |
|좀        |236  |
|ㅋㅋ      |224  |
|빨대      |218  |
|종이컵    |208  |
|종이빨대  |203  |
|환경      |185  |
|컵        |185  |
|하는      |184  |
|그럼      |167  |
|종이      |160  |
|수        |156  |
|ㅋㅋㅋ    |148  |
|일회용    |148  |
|안        |147  |
|너무      |143  |
|하면      |142  |
|많이      |137  |
|재활용    |137  |
|그        |135  |
|잘        |128  |
|하고      |124  |
|쓰레기    |119  |
|또        |116  |
|이제      |114  |
|할        |111  |
|근데      |110  |
|텀블러    |110  |
|이        |110  |
|아니      |109  |
|설거지    |108  |
|종이컵은  |108  |
|있는      |108  |
|규제      |106  |
|어떻게    |105  |
|이런      |103  |
|뭐        |101  |
|무슨      |100  |
|친환경    |100  |
|환경을    |98   |
|그리고    |96   |
|지금      |93   |
|제대로    |92   |
|이게      |90   |
|종이빨대는|88   |
|것        |85   |
|정말      |85   |
|빨대는    |84   |
|없는      |84   |

In [51]:
# 좋아요 수 대로 정렬

from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

# SparkSession 초기화
spark = SparkSession.builder \
    .appName("Likes Sorting and Comment Display") \
    .getOrCreate()

# plastic_clean_data.csv 파일을 읽어서 DataFrame 생성
file_path = "assembled_data - assembled_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# null 값을 가진 행 제거
df = df.dropna(subset=["comment"])

# likes 컬럼을 기준으로 내림차순 정렬
sorted_df = df.orderBy(desc("num_likes"))

# 정렬된 결과에서 comment와 likes 컬럼을 선택하여 출력
sorted_df.select("comment", "num_likes").show(truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|comment                                                                                                                                                                                                                                                                                               |num_likes|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|환경을 생각하는건 좋은데 왜 서민들부터 제재하지 대기업들부터 과대포장안하고 물티슈나 일회용품부터 자연분해될수 있는것들로 만들면 되잖아 이것만

In [57]:
# 작성날짜 (오래된순, 최신순) 별 단어빈도수 분석

from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import explode, desc

# SparkSession 초기화
spark = SparkSession.builder \
    .appName("Word Frequency and Date Analysis") \
    .getOrCreate()

# assembled_data.csv 파일을 읽어서 DataFrame 생성
file_path = "assembled_data - assembled_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# null 값을 가진 행 제거
df = df.dropna(subset=["comment"])

# 최소 2023년 이후의 데이터 중 가장 오래된 100개의 comment를 선택
oldest_comments_after_2023 = df.filter(df["date"] >= "2023-01-01").orderBy("date").limit(1000)

# 최신 2023년 이후의 데이터 중 가장 최근 100개의 comment를 선택
latest_comments_after_2023 = df.filter(df["date"] >= "2023-01-01").orderBy(desc("date")).limit(1000)

# Tokenizer를 사용하여 단어 추출
tokenizer = Tokenizer(inputCol="comment", outputCol="words")

# 가장 오래된 100개의 comment에 대한 단어 빈도수 분석
tokenized_df_oldest = tokenizer.transform(oldest_comments_after_2023)
word_count_df_oldest = tokenized_df_oldest.select(explode("words").alias("word")).groupBy("word").count()
sorted_word_count_df_oldest = word_count_df_oldest.orderBy(desc("count")).limit(50)

# 가장 최근 100개의 comment에 대한 단어 빈도수 분석
tokenized_df_latest = tokenizer.transform(latest_comments_after_2023)
word_count_df_latest = tokenized_df_latest.select(explode("words").alias("word")).groupBy("word").count()
sorted_word_count_df_latest = word_count_df_latest.orderBy(desc("count")).limit(50)

# 결과를 출력
print("2023년 이후 가장 오래된 100개의 Comment:")
oldest_comments_after_2023.select("date", "comment").show(100, truncate=False)

print("\nTop 50 단어 빈도수 (가장 오래된 100개 Comment, 2023년 이후):")
sorted_word_count_df_oldest.show(n=50, truncate=False)

print("\n2023년 이후 가장 최근 100개의 Comment:")
latest_comments_after_2023.select("date", "comment").show(100, truncate=False)

print("\nTop 50 단어 빈도수 (가장 최근 100개 Comment, 2023년 이후):")
sorted_word_count_df_latest.show(n=50, truncate=False)


2023년 이후 가장 오래된 100개의 Comment:
+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|date      |comment                                                                                                                                                                                                                                                                                                                                                                                                             |
+----------+-------------------------------------------------------------------------------------------------------------------------

In [69]:
# 겹치는거제외한 단어빈도수(오래된 순, 최신순)
# 가장 오래된 100개 Comment와 가장 최근 100개 Comment의 단어 빈도수 비교
unique_words_oldest = set(sorted_word_count_df_oldest.select("word").rdd.flatMap(lambda x: x).collect())
unique_words_latest = set(sorted_word_count_df_latest.select("word").rdd.flatMap(lambda x: x).collect())

# 각 데이터셋에서만 등장하는 단어 추출
unique_words_in_oldest = unique_words_oldest - unique_words_latest
unique_words_in_latest = unique_words_latest - unique_words_oldest

# 각 데이터셋에서만 등장하는 단어의 빈도수 추출
unique_word_count_oldest = word_count_df_oldest.filter(word_count_df_oldest["word"].isin(list(unique_words_in_oldest)))
unique_word_count_latest = word_count_df_latest.filter(word_count_df_latest["word"].isin(list(unique_words_in_latest)))

# 결과 출력
print("\n가장 오래된 100개 Comment에서만 등장하는 단어와 빈도수:")
unique_word_count_oldest.show(truncate=False)

print("\n가장 최근 100개 Comment에서만 등장하는 단어와 빈도수:")
unique_word_count_latest.show(truncate=False)



가장 오래된 100개 Comment에서만 등장하는 단어와 빈도수:
+----------+-----+
|word      |count|
+----------+-----+
|컵        |35   |
|이쑤시개  |15   |
|이런      |17   |
|얼마나    |16   |
|무조건    |15   |
|하면      |24   |
|재활용    |19   |
|설거지    |28   |
|그리고    |21   |
|ㅋㅋㅋ    |17   |
|이        |17   |
|다른      |17   |
|쓰레기    |14   |
|환경단체  |24   |
|참        |15   |
|종이컵은  |19   |
|환경에    |14   |
|종이빨대는|21   |
+----------+-----+


가장 최근 100개 Comment에서만 등장하는 단어와 빈도수:
+----------+-----+
|word      |count|
+----------+-----+
|정말      |17   |
|커피      |14   |
|정책      |15   |
|한        |12   |
|이게      |13   |
|없는      |17   |
|정부      |15   |
|텀블러    |18   |
|뭔        |13   |
|종이빨대가|19   |
|빨대가    |14   |
|다시      |14   |
|제대로    |14   |
|환경을    |13   |
|것        |12   |
|?         |14   |
|있는      |17   |
|문재인    |16   |
+----------+-----+



In [58]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import explode, desc

# SparkSession 초기화
spark = SparkSession.builder \
    .appName("Word Frequency and Date Analysis") \
    .getOrCreate()

# assembled_data.csv 파일을 읽어서 DataFrame 생성
file_path = "assembled_data - assembled_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# null 값을 가진 행 제거
df = df.dropna(subset=["comment"])2ㅈ

# 좋아요 수(num_likes)가 가장 많은 순으로 데이터 정렬
df_sorted_by_likes = df.orderBy(desc("num_likes")).limit(100)

# Tokenizer를 사용하여 단어 추출
tokenizer = Tokenizer(inputCol="comment", outputCol="words")

# 좋아요 수가 가장 많은 댓글에 대한 단어 빈도수 분석
tokenized_df_top_likes = tokenizer.transform(df_sorted_by_likes)
word_count_df_top_likes = tokenized_df_top_likes.select(explode("words").alias("word")).groupBy("word").count()
sorted_word_count_df_top_likes = word_count_df_top_likes.orderBy(desc("count")).limit(50)

# 결과를 출력
print("좋아요 수가 가장 많은 100개의 Comment:")
df_sorted_by_likes.select("num_likes", "comment").show(100, truncate=False)

print("\nTop 50 단어 빈도수 (좋아요 수가 가장 많은 100개 Comment):")
sorted_word_count_df_top_likes.show(n=50, truncate=False)


좋아요 수가 가장 많은 100개의 Comment:
+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|num_likes|comment                                                                                                                                                                                                                                                                                                                                                                           

In [66]:
from pyspark.sql.functions import col

# Instagram 태그에 대한 단어 빈도수 분석
instagram_comments = df.filter(col("tag") == "instagram").filter(df["date"] >= "2023-01-01").orderBy("date").limit(1000)
tokenized_df_instagram = tokenizer.transform(instagram_comments)
word_count_df_instagram = tokenized_df_instagram.select(explode("words").alias("word")).groupBy("word").count()
sorted_word_count_df_instagram = word_count_df_instagram.orderBy(desc("count")).limit(50)

# News 태그에 대한 단어 빈도수 분석
news_comments = df.filter(col("tag") == "news").filter(df["date"] >= "2023-01-01").orderBy("date").limit(1000)
tokenized_df_news = tokenizer.transform(news_comments)
word_count_df_news = tokenized_df_news.select(explode("words").alias("word")).groupBy("word").count()
sorted_word_count_df_news = word_count_df_news.orderBy(desc("count")).limit(50)

# YouTube 태그에 대한 단어 빈도수 분석
youtube_comments = df.filter(col("tag") == "youtube").filter(df["date"] >= "2023-01-01").orderBy("date").limit(1000)
tokenized_df_youtube = tokenizer.transform(youtube_comments)
word_count_df_youtube = tokenized_df_youtube.select(explode("words").alias("word")).groupBy("word").count()
sorted_word_count_df_youtube = word_count_df_youtube.orderBy(desc("count")).limit(50)

# 결과를 출력
print("\nInstagram 태그 Comment의 Top 50 단어 빈도수 (2023년 이후):")
sorted_word_count_df_instagram.show(n=50, truncate=False)

print("\nNews 태그 Comment의 Top 50 단어 빈도수 (2023년 이후):")
sorted_word_count_df_news.show(n=50, truncate=False)

print("\nYouTube 태그 Comment의 Top 50 단어 빈도수 (2023년 이후):")
sorted_word_count_df_youtube.show(n=50, truncate=False)



Instagram 태그 Comment의 Top 50 단어 빈도수 (2023년 이후):
+----------+-----+
|word      |count|
+----------+-----+
|          |759  |
|종이빨대  |79   |
|진짜      |50   |
|빨대      |48   |
|플라스틱  |46   |
|더        |35   |
|다        |33   |
|그럼      |31   |
|종이      |30   |
|왜        |30   |
|그냥      |28   |
|좀        |23   |
|텀블러    |23   |
|또        |21   |
|이제      |20   |
|일회용품  |20   |
|뭐        |18   |
|어떻게    |18   |
|종이빨대가|17   |
|너무      |15   |
|하는      |15   |
|정말      |15   |
|커피      |14   |
|정부      |14   |
|아니      |13   |
|정책      |13   |
|종이빨대는|13   |
|다시      |13   |
|ㅋㅋ      |12   |
|잘        |12   |
|친환경    |12   |
|아        |12   |
|이랬다    |12   |
|테이크아웃|12   |
|일회용    |11   |
|환경      |11   |
|많이      |11   |
|종이컵    |10   |
|누가      |10   |
|제대로    |10   |
|환경을    |10   |
|이건      |9    |
|이거      |9    |
|이런      |9    |
|이게      |9    |
|빨대를    |9    |
|저랬다    |9    |
|저랬다가  |9    |
|이랬다가  |9    |
|걍        |9    |
+----------+-----+


News 태그 Comment의 Top 50 단어 빈도수 (2023년 이후)

In [68]:
from pyspark.sql.functions import col, explode

# Instagram, News, YouTube 각 태그에 대한 단어 목록을 수집
instagram_words = tokenized_df_instagram.select("words").rdd.flatMap(lambda x: x[0]).collect()
news_words = tokenized_df_news.select("words").rdd.flatMap(lambda x: x[0]).collect()
youtube_words = tokenized_df_youtube.select("words").rdd.flatMap(lambda x: x[0]).collect()

# 각 태그에서만 등장하는 유일한 단어를 찾기
unique_instagram_words = set(instagram_words) - set(news_words) - set(youtube_words)
unique_news_words = set(news_words) - set(instagram_words) - set(youtube_words)
unique_youtube_words = set(youtube_words) - set(instagram_words) - set(news_words)

# 각 태그에 대한 DataFrame 생성
instagram_df = tokenized_df_instagram.withColumn("unique_words", explode("words")).filter(col("unique_words").isin(list(unique_instagram_words)))
news_df = tokenized_df_news.withColumn("unique_words", explode("words")).filter(col("unique_words").isin(list(unique_news_words)))
youtube_df = tokenized_df_youtube.withColumn("unique_words", explode("words")).filter(col("unique_words").isin(list(unique_youtube_words)))

# 각 태그에서 유일한 단어의 빈도수 계산
instagram_word_count_df = instagram_df.groupBy("unique_words").count().orderBy(desc("count")).limit(50)
news_word_count_df = news_df.groupBy("unique_words").count().orderBy(desc("count")).limit(50)
youtube_word_count_df = youtube_df.groupBy("unique_words").count().orderBy(desc("count")).limit(50)

# 결과 출력
print("\nInstagram 태그에서만 등장하는 유일한 단어의 빈도수:")
instagram_word_count_df.show(truncate=False)

print("\nNews 태그에서만 등장하는 유일한 단어의 빈도수:")
news_word_count_df.show(truncate=False)

print("\nYouTube 태그에서만 등장하는 유일한 단어의 빈도수:")
youtube_word_count_df.show(truncate=False)



Instagram 태그에서만 등장하는 유일한 단어의 빈도수:
+------------+-----+
|unique_words|count|
+------------+-----+
|이랬다      |12   |
|저랬다      |9    |
|저랬다가    |9    |
|이랬다가    |9    |
|이랬다저랬다|8    |
|오락가락    |7    |
|카페들      |6    |
|정책이      |5    |
|거꾸로      |5    |
|스벅은      |5    |
|엥          |5    |
|손바닥      |4    |
|아휴        |4    |
|손으로      |4    |
|사실        |4    |
|화가        |4    |
|시대를      |4    |
|좋던데      |4    |
|왔다갔다    |4    |
|기후위기    |4    |
+------------+-----+
only showing top 20 rows


News 태그에서만 등장하는 유일한 단어의 빈도수:
+-------------+-----+
|unique_words |count|
+-------------+-----+
|환경만       |6    |
|법카로       |5    |
|한마디       |5    |
|한다.        |4    |
|자연으로     |4    |
|언론에       |4    |
|""           |4    |
|일본산       |4    |
|입이         |4    |
|자가         |4    |
|아니다.      |4    |
|선택적       |4    |
|//           |4    |
|불편했고     |4    |
|부득이한     |4    |
|조치였다     |4    |
|폐해도       |4    |
|많았다.      |4    |
|환경파괴다...|4    |
|텁텁한       |4    |
+-------------+----